In [1]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from copy import deepcopy

# ==========================================
# 1. THE ENVIRONMENT
# ==========================================
class Connect4:
    def __init__(self, rows=6, cols=7, connect=4):
        self.rows = rows
        self.cols = cols
        self.connect = connect
        self.reset()

    def reset(self):
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.current_player = 1  # 1 = Agent, -1 = Opponent
        self.done = False
        self.winner = None
        return self.board.copy()

    def valid_actions(self):
        return [c for c in range(self.cols) if self.board[0, c] == 0]

    def step(self, action):
        if action not in self.valid_actions():
            raise ValueError(f"Invalid action {action}")

        # Drop piece
        for r in range(self.rows - 1, -1, -1):
            if self.board[r, action] == 0:
                self.board[r, action] = self.current_player
                break

        # Check winner
        winner = self.check_winner(self.board)
        reward = 0
        
        if winner is not None:
            self.done = True
            self.winner = winner
            # Reward: 1 if current_player won, -1 if they lost
            reward = 1 if winner == self.current_player else -1
        elif np.all(self.board != 0):
            self.done = True
            self.winner = 0
            reward = 0 # Draw
        
        # Switch player if game not over
        if not self.done:
            self.current_player *= -1
            
        return self.board.copy(), reward, self.done, {}

    def check_winner(self, board):
        # Optimized check using sliding windows
        rows, cols = board.shape
        # Horizontal
        for r in range(rows):
            for c in range(cols - 3):
                window = board[r, c:c+4]
                if abs(sum(window)) == 4 and np.all(window != 0):
                    return window[0]
        # Vertical
        for c in range(cols):
            for r in range(rows - 3):
                window = board[r:r+4, c]
                if abs(sum(window)) == 4 and np.all(window != 0):
                    return window[0]
        # Diagonals
        for r in range(rows - 3):
            for c in range(cols - 3):
                # Down-right
                w1 = [board[r+i, c+i] for i in range(4)]
                if abs(sum(w1)) == 4 and all(x != 0 for x in w1): return w1[0]
                # Up-right
                w2 = [board[r+3-i, c+i] for i in range(4)]
                if abs(sum(w2)) == 4 and all(x != 0 for x in w2): return w2[0]
        return None

# ==========================================
# 2. HELPER: MINIMAX OPPONENT (For Training)
# ==========================================
def evaluate_window(window, piece):
    score = 0
    opp_piece = -piece
    if window.count(piece) == 4: score += 100
    elif window.count(piece) == 3 and window.count(0) == 1: score += 5
    elif window.count(piece) == 2 and window.count(0) == 2: score += 2
    if window.count(opp_piece) == 3 and window.count(0) == 1: score -= 4
    return score

def score_position(board, piece):
    score = 0
    # Center column preference
    center_array = [int(i) for i in list(board[:, 7//2])]
    center_count = center_array.count(piece)
    score += center_count * 3
    
    # Horizontal, Vertical, Diagonal checks...
    # (Simplified for brevity, assumes standard connect 4 logic)
    # Ideally, reuse the feature logic, but Minimax needs a scalar score.
    return score # Placeholder for full heuristic

def minimax(board, depth, alpha, beta, maximizingPlayer, piece):
    valid_locations = [c for c in range(7) if board[0,c] == 0]
    is_terminal = (Connect4().check_winner(board) is not None) or len(valid_locations) == 0
    
    if depth == 0 or is_terminal:
        if is_terminal:
            winner = Connect4().check_winner(board)
            if winner == piece: return 100000000000000, None
            elif winner == -piece: return -10000000000000, None
            else: return 0, None
        else:
            return score_position(board, piece), None

    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            b_copy = board.copy()
            # Drop piece
            for r in range(5, -1, -1):
                if b_copy[r, col] == 0:
                    b_copy[r, col] = piece
                    break
            new_score, _ = minimax(b_copy, depth-1, alpha, beta, False, piece)
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta: break
        return value, column
    else:
        value = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            b_copy = board.copy()
            # Drop piece
            for r in range(5, -1, -1):
                if b_copy[r, col] == 0:
                    b_copy[r, col] = -piece
                    break
            new_score, _ = minimax(b_copy, depth-1, alpha, beta, True, piece)
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta: break
        return value, column

# ==========================================
# 3. THE IMPROVED AGENT (Linear Approx)
# ==========================================

class LinearAgent:
    def __init__(self, num_features=6, alpha=0.01, gamma=0.9):
        # Weights: [Bias, My3, My2, Opp3, Opp2, Center]
        self.weights = np.zeros(num_features)
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.99995

    def get_features(self, board, player):
        """
        Extract features from the board relative to 'player'.
        """
        opp = -player
        rows, cols = board.shape
        
        my_3 = 0
        my_2 = 0
        opp_3 = 0
        opp_2 = 0
        
        # Helper to analyze a window of 4 cells
        def check(window):
            nonlocal my_3, my_2, opp_3, opp_2
            cnt_p = np.count_nonzero(window == player)
            cnt_o = np.count_nonzero(window == opp)
            cnt_e = np.count_nonzero(window == 0)
            
            if cnt_p == 3 and cnt_e == 1: my_3 += 1
            elif cnt_p == 2 and cnt_e == 2: my_2 += 1
            
            if cnt_o == 3 and cnt_e == 1: opp_3 += 1
            elif cnt_o == 2 and cnt_e == 2: opp_2 += 1

        # Scan board (Horiz, Vert, Diag)
        # Horizontal
        for r in range(rows):
            for c in range(cols - 3):
                check(board[r, c:c+4])
        # Vertical
        for c in range(cols):
            for r in range(rows - 3):
                check(board[r:r+4, c])
        # Diagonals
        for r in range(rows - 3):
            for c in range(cols - 3):
                check([board[r+i, c+i] for i in range(4)])
                check([board[r+3-i, c+i] for i in range(4)])

        # Center Control
        center_col = board[:, cols//2]
        center_ctrl = np.count_nonzero(center_col == player)

        # Feature Vector: [Bias, My3, My2, Opp3, Opp2, Center]
        return np.array([1.0, my_3, my_2, opp_3, opp_2, center_ctrl])

    def evaluate(self, board, player):
        features = self.get_features(board, player)
        return np.dot(self.weights, features)

    def choose_action(self, env, training=True):
        valid = env.valid_actions()
        if not valid: return None

        # Epsilon Greedy
        if training and random.random() < self.epsilon:
            return random.choice(valid)

        # Greedy based on Afterstate Value
        best_val = -float('inf')
        best_act = random.choice(valid)

        for act in valid:
            # Simulate Next State
            temp_board = env.board.copy()
            for r in range(env.rows - 1, -1, -1):
                if temp_board[r, act] == 0:
                    temp_board[r, act] = env.current_player
                    break
            
            # Check immediate win (Manual check to encourage winning)
            if Connect4().check_winner(temp_board) == env.current_player:
                return act

            # Evaluate state VALUE
            val = self.evaluate(temp_board, env.current_player)
            if val > best_val:
                best_val = val
                best_act = act
        
        return best_act

    def update(self, prev_board, action, reward, next_board, done, player):
        """
        Gradient Descent Update:
        Target = Reward + Gamma * V(next_best_state)
        Error = Target - V(current_afterstate)
        Weights += Alpha * Error * Features
        """
        # 1. Feature of the state we just created (The "Afterstate")
        # We need to reconstruct the board resulting from prev_board + action
        after_board = prev_board.copy()
        for r in range(6 - 1, -1, -1):
            if after_board[r, action] == 0:
                after_board[r, action] = player
                break
        
        features = self.get_features(after_board, player)
        current_val = np.dot(self.weights, features)

        if done:
            target = reward
        else:
            # Bootstrap: Value of the board after opponent moves and we move again? 
            # Simplified: Value of the resulting board geometry directly
            target = reward + self.gamma * self.evaluate(next_board, player)

        error = target - current_val
        self.weights += self.alpha * error * features

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


# ==========================================
# 4. TRAINING LOOP
# ==========================================
def train(episodes=5000):
    agent = LinearAgent()
    env = Connect4()
    wins = 0
    losses = 0
    
    print(f"Training for {episodes} episodes...")
    
    for ep in range(1, episodes+1):
        env.reset()
        # Randomize start order
        agent_p = 1
        opp_p = -1
        
        # If we want agent to practice going second, flip logic occasionally
        if random.random() < 0.5:
            # Agent is player 2 (-1), but internal logic assumes current_player
            # For simplicity, let's keep Agent as Player 1 (1) in the env, 
            # but let Opponent move first sometimes.
            # Opponent Move 1:
            opp_act = random.choice(env.valid_actions())
            env.step(opp_act)
            
        done = False
        while not done:
            # --- AGENT TURN ---
            state_before = env.board.copy()
            action = agent.choose_action(env)
            
            _, reward, done, _ = env.step(action)
            state_after = env.board.copy()

            if done:
                # Agent Won (Reward 1) or Draw (0)
                agent.update(state_before, action, reward, state_after, done, 1)
                if reward == 1: wins += 1
                break

            # --- OPPONENT TURN ---
            # Use Minimax (depth 1 or 2) for decent competition
            # Using random occasionally helps exploration
            if random.random() < 0.3:
                 opp_action = random.choice(env.valid_actions())
            else:
                 _, opp_action = minimax(env.board, 2, -math.inf, math.inf, False, -1)
            
            if opp_action is None: opp_action = random.choice(env.valid_actions()) # Fallback
            
            _, reward_opp, done, _ = env.step(opp_action)
            state_final = env.board.copy()

            # If opponent won, reward_opp is -1 (from env perspective of who won).
            # But relative to agent, opponent winning is a massive negative.
            if done:
                r = -1 if env.winner == -1 else 0
                agent.update(state_before, action, r, state_final, done, 1)
                if r == -1: losses += 1
            else:
                # Game continues. Update agent based on state AFTER opponent moved
                agent.update(state_before, action, 0, state_final, done, 1)

        if ep % 500 == 0:
            print(f"Ep {ep}: Wins {wins}, Losses {losses}, Epsilon {agent.epsilon:.3f}")
            print(f"Weights: {np.round(agent.weights, 3)}")
            wins = 0; losses = 0

    return agent

# ==========================================
# 5. EXECUTION
# ==========================================

# 1. Train the Agent
trained_agent = train(episodes=3000)

# 2. Play a Game (Human vs AI)
print("\n--- READY TO PLAY ---")
env = Connect4()
game_over = False
env.render = lambda: print(env.board) # Simple render

while not game_over:
    print("\nCurrent Board:")
    env.render()
    
    # Human Move
    valid = env.valid_actions()
    try:
        col = int(input(f"Your move (0-6) {valid}: "))
    except:
        col = valid[0]
        
    if col in valid:
        _, _, game_over, _ = env.step(col)
        if game_over:
            env.render()
            if env.winner == 1: print("Human Wins!")
            else: print("Draw!")
            break
            
        # AI Move
        print("AI Thinking...")
        # Disable epsilon for actual play
        trained_agent.epsilon = 0 
        col = trained_agent.choose_action(env, training=False)
        _, _, game_over, _ = env.step(col)
        
        if game_over:
            env.render()
            if env.winner == -1: print("AI Wins!")
            else: print("Draw!")
            break

Training for 3000 episodes...
Ep 500: Wins 108, Losses 171, Epsilon 0.803
Weights: [-0.09   0.064  0.009 -0.14  -0.005 -0.02 ]
Ep 1000: Wins 184, Losses 157, Epsilon 0.642
Weights: [-0.079  0.055 -0.021 -0.168 -0.049 -0.012]
Ep 1500: Wins 226, Losses 104, Epsilon 0.518
Weights: [ 0.028  0.079 -0.014  0.016  0.017  0.016]
Ep 2000: Wins 235, Losses 125, Epsilon 0.407
Weights: [ 0.058  0.021 -0.032 -0.137 -0.019 -0.007]
Ep 2500: Wins 257, Losses 101, Epsilon 0.322
Weights: [ 0.041  0.09   0.016 -0.073 -0.011  0.046]
Ep 3000: Wins 261, Losses 90, Epsilon 0.258
Weights: [ 0.048  0.103 -0.017 -0.073 -0.032  0.009]

--- READY TO PLAY ---

Current Board:
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
AI Thinking...

Current Board:
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]]
AI Thinking...

Current Board:
[[ 0  0  0  0  0  0  0]
 [ 0  0  0 